# 데이터사이언스융합학과 2024710658 박창준

# 트리의 앙상블

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-3.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 랜덤포레스트

In [ ]:
# 데이터 불러오기
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 온라인 링크로 데이터 불러오기 가능
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# x 데이터 설정
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# y 데이터 설정
target = wine['class'].to_numpy()

# train_test_split 함수 이용해서 train x, test x, train y, test y 데이터로 분리
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# 교차검증을 하기 위한 cross_validate 불러오기
from sklearn.model_selection import cross_validate
# RandomForestClassifier 불러오기
from sklearn.ensemble import RandomForestClassifier

# RandomForestClassifier 객체 생성
# n_jobs = -1 : 로컬의 CPU 자원을 사용할 경우에 적용하는 파라미터
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 학습 스코어가 너무 높음 -> 오버피팅 발생 -> 파라미터 변경 / 모델 변경 등 조치 필요

0.9973541965122431 0.8905151032797809


In [ ]:
# 생성한 rf 객체 학습
rf.fit(train_input, train_target)
# 학습한 rf 모델에서의 feature_importances_ 결과 출력
print(rf.feature_importances_)
# Decision Tree에서보다 중요도가 덜 치우친 것 확인 가능
# 데이터를 랜덤하게 샘플링 했기 때문 & 앙상블 모델의 효과 -> 더 많은 피처를 훈련할 수 있음 -> 과대적합 감소 및 성능 향상

[0.23167441 0.50039841 0.26792718]


In [ ]:
# RandomForestClassifier 객체 생성
# oob_score = True : Out of bags (훈련에서 사용하지 않은 샘플을 의미)
# 중복을 허용하여 데이터 subset을 구성하기 때문에, 사용하지 않은 데이터가 존재할 수 있음
# 이러한 데이터를 검증에 활용
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

# 생성한 rf 객체 학습
rf.fit(train_input, train_target)
# oob를 활용한 검증 스코어 출력
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [ ]:
# ExtraTreesClassifier 불러오기
# ExtraTreesRegressor : 엑스트라 트리 회귀 모델(참고)
from sklearn.ensemble import ExtraTreesClassifier

# ExtraTreesClassifier 객체 생성
# n_jobs = -1 : 로컬의 CPU 자원을 사용할 경우에 적용하는 파라미터
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# rf와 비슷한 결과 : 학습 스코어가 너무 높음 -> 오버피팅

0.9974503966084433 0.8887848893166506


In [ ]:
# 생성한 et 객체 학습
et.fit(train_input, train_target)
# 학습한 et 모델에서의 feature_importances_ 결과 출력
print(et.feature_importances_)
# 역시나 rf와 비슷한 결과 나옴

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [ ]:
# GradientBoostingClassifier 불러오기
from sklearn.ensemble import GradientBoostingClassifier

# GradientBoostingClassifier 객체 생성
gb = GradientBoostingClassifier(random_state=42)

# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# rf와 et에서의 오버피팅이 해결됨 -> 깊이가 얕은 트리를 사용했기 때문

0.8881086892152563 0.8720430147331015


In [ ]:
# GradientBoostingClassifier 객체 생성
# 성능 향상을 위해 n_estimators 변경 (100 -> 500), learning_rate 변경 (0.1 -> 0.2)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 학습 스코어와 테스트 스코어 모두 약간 향상됨

0.9464595437171814 0.8780082549788999


In [ ]:
# 생성한 gb 객체 학습
gb.fit(train_input, train_target)
# 학습한 gb 모델에서의 feature_importances_ 결과 출력
print(gb.feature_importances_)
# rf나 et보다 중요한 피처에 집중하는 경향 확인
# 이건 값의 잔차 보완에 집중하는 과정에서 중요한 피처의 영향 커짐

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅

#### XGBoost

In [ ]:
# XGBClassifier 불러오기
from xgboost import XGBClassifier

# XGBClassifier 객체 생성
# tree_method='hist' : 데이터를 구간으로 묶어서, 그 안에서 best split 찾는 것을 의미
# tree_method='auto' : 데이터 개숭 따라 자동으로 설정
# tree_method='exact' : 데이터 전수조사를 의미
# tree_method='gpu_hist' : gpu를 사용해 hist 방식 적용 -> 속도가 더 빠름
# tree_method='approx' : hist와 유사한 방식에 약간의 가중치를 고려한 방식
xgb = XGBClassifier(tree_method='hist', random_state=42)

# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [ ]:
# LGBMClassifier 불러오기
from lightgbm import LGBMClassifier

# LGBMClassifier 객체 생성
lgb = LGBMClassifier(random_state=42)
# cross_validate 객체 생성
# cross_validate(객체명, train x, train y, 기타 파라미터)
# return_train_score : 훈련점수와 검증점수를 모두 반환하는 파라미터
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# 학습 스코어와 테스트 스코어 출력
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
